In [1]:
import pandas as pd
import datetime
import urllib
import os
import glob

In [6]:
for i in range(1,13):
    for j in ['fhv','green','yellow']
        if len(str(i)) == 1:
            urllib.request.urlretrieve('https://s3.amazonaws.com/nyc-tlc/trip+data/%j_tripdata_2018-0%s.csv'%(j,i),'transportation/NewYork/%j_tripdata_2018-0%s.csv'%(j,i))
        elif len(str(i)) == 2:
            urllib.request.urlretrieve('https://s3.amazonaws.com/nyc-tlc/trip+data/%j_tripdata_2018-%s.csv'%(j,i),'transportation/NewYork/%j_tripdata_2018-%s.csv'%(j,i))
        print(j,i)

1
2
3
4
5
6
7
8
9
10
11
12


### Writte different months taxi data into one csv file

In [2]:
read_path = '/Users/hemingyi/Documents/capstone/transportation/NewYork/*.csv'
save_path = '/Users/hemingyi/Documents/capstone/transportation/NewYork/compacted/NewYork_Taxi.csv'

In [10]:
df = pd.read_csv(glob.glob(read_path)[0])  
df = df.dropna(subset=['Pickup_DateTime','PUlocationID', 'DOlocationID'])
df['PUlocationID'] = df['PUlocationID'].astype('int')
df['DOlocationID'] = df['DOlocationID'].astype('int')
df['Pickup_DateTime'] = pd.to_datetime(df['Pickup_DateTime'])
df['Date'] = df['Pickup_DateTime'].dt.strftime('%m/%d/%Y')
df['PassengerAmount'] = 1
df.rename(columns={'PUlocationID':'PickupLocationID','DOlocationID':'DropoffLocationID'},inplace=True)
df['flag'] = (glob.glob(read_path)[0]).split('/')[-1].split('_')[0]
df = df[['Date','PickupLocationID','DropoffLocationID','PassengerAmount','flag']]
df.to_csv(save_path,index=False)

for i in glob.glob(read_path)[1:]:
    print(i)
    df = pd.read_csv(i)
    if str(i).split('/')[-1].split('_')[0] == 'yellow':
#         print('yellow')
        df = df.dropna(subset=['tpep_pickup_datetime','PULocationID', 'DOLocationID'])
        df['PULocationID'] = df['PULocationID'].astype('int')
        df['DOLocationID'] = df['DOLocationID'].astype('int')
        #no zone name is numbered as 0 and 264, 265 refer to NA
        df = df[(df['PULocationID']!=0)&(df['DOLocationID']!=0)]
        df = df[(df['PULocationID']!=264)&(df['DOLocationID']!=264)]
        df = df[(df['PULocationID']!=265)&(df['DOLocationID']!=265)]
        df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
        df['Date'] = df['tpep_pickup_datetime'].dt.strftime('%m/%d/%Y')
        df.rename(columns={'PULocationID':'PickupLocationID','DOLocationID':'DropoffLocationID','passenger_count':'PassengerAmount'},inplace=True)
    elif str(i).split('/')[-1].split('_')[0] == 'green':
#         print('green')
        df = df.dropna(subset=['lpep_pickup_datetime','PULocationID', 'DOLocationID'])
        df['PULocationID'] = df['PULocationID'].astype('int')
        df['DOLocationID'] = df['DOLocationID'].astype('int')
        df = df[(df['PULocationID']!=0)&(df['DOLocationID']!=0)]
        df = df[(df['PULocationID']!=264)&(df['DOLocationID']!=264)]
        df = df[(df['PULocationID']!=265)&(df['DOLocationID']!=265)]
        df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
        df['Date'] = df['lpep_pickup_datetime'].dt.strftime('%m/%d/%Y')
        df.rename(columns={'PULocationID':'PickupLocationID','DOLocationID':'DropoffLocationID','passenger_count':'PassengerAmount'},inplace=True)
    elif str(i).split('/')[-1].split('_')[0] == 'fhv':
#         print('fhv')
        df = df.dropna(subset=['Pickup_DateTime','PUlocationID', 'DOlocationID'])
        df['PUlocationID'] = df['PUlocationID'].astype('int')
        df['DOlocationID'] = df['DOlocationID'].astype('int')
        df = df[(df['PUlocationID']!=0)&(df['DOlocationID']!=0)]
        df = df[(df['PUlocationID']!=264)&(df['DOlocationID']!=264)]
        df = df[(df['PUlocationID']!=265)&(df['DOlocationID']!=265)]
        df['Pickup_DateTime'] = pd.to_datetime(df['Pickup_DateTime'])
        df['Date'] = df['Pickup_DateTime'].dt.strftime('%m/%d/%Y')
        df['PassengerAmount'] = 1
        df.rename(columns={'PUlocationID':'PickupLocationID','DOlocationID':'DropoffLocationID'},inplace=True)
    df['flag'] = str(i).split('/')[-1].split('_')[0]
    print(df.columns)
    df = df[['Date','PickupLocationID','DropoffLocationID','PassengerAmount','flag']]
    df.to_csv(save_path,index=False, header=False, mode='a+')

/Users/hemingyi/Documents/capstone/transportation/NewYork/green_tripdata_2017-08.csv
Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PickupLocationID',
       'DropoffLocationID', 'PassengerAmount', 'trip_distance', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee',
       'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type',
       'Date', 'flag'],
      dtype='object')
/Users/hemingyi/Documents/capstone/transportation/NewYork/yellow_tripdata_2017-01.csv
Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'PassengerAmount', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PickupLocationID', 'DropoffLocationID', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'Date', 'flag'],
      dtype='object')
/Users/hemingyi/Documents/capstone/transportation/NewYor

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PickupLocationID',
       'DropoffLocationID', 'PassengerAmount', 'trip_distance', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee',
       'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type',
       'Date', 'flag'],
      dtype='object')
/Users/hemingyi/Documents/capstone/transportation/NewYork/green_tripdata_2018-04.csv
Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PickupLocationID',
       'DropoffLocationID', 'PassengerAmount', 'trip_distance', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee',
       'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type',
       'Date', 'flag'],
      dtype='object')
/Users/hemingyi/Documents/capstone/transportation/NewYork/green_tripdata_2018-10.csv
Index(['VendorID', 'lpep

/Users/hemingyi/Documents/capstone/transportation/NewYork/fhv_tripdata_2018-02.csv
Index(['Pickup_DateTime', 'DropOff_datetime', 'PickupLocationID',
       'DropoffLocationID', 'SR_Flag', 'Dispatching_base_number',
       'Dispatching_base_num', 'Date', 'PassengerAmount', 'flag'],
      dtype='object')
/Users/hemingyi/Documents/capstone/transportation/NewYork/fhv_tripdata_2018-03.csv
Index(['Pickup_DateTime', 'DropOff_datetime', 'PickupLocationID',
       'DropoffLocationID', 'SR_Flag', 'Dispatching_base_number',
       'Dispatching_base_num', 'Date', 'PassengerAmount', 'flag'],
      dtype='object')
/Users/hemingyi/Documents/capstone/transportation/NewYork/green_tripdata_2018-08.csv
Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PickupLocationID',
       'DropoffLocationID', 'PassengerAmount', 'trip_distance', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee',
       'improvement_surchar